In [4]:
# PyTorch 및 기타 라이브러리 임포트
import torch  # PyTorch 텐서 및 연산을 위한 라이브러리
import torch.nn as nn  # 신경망 모델을 정의하기 위한 모듈
import torch.optim as optim  # 최적화 알고리즘을 위한 모듈 (예: Adam, SGD)
import torch.nn.functional as F  # 신경망에서 자주 사용되는 활성화 함수들 (ReLU 등)
from torch.utils.data import DataLoader, TensorDataset  # 데이터셋을 배치로 처리하기 위한 모듈
import pandas as pd  # 데이터프레임을 다루는 라이브러리 (CSV 파일 처리 등)
import numpy as np  # 수학적 연산 및 배열 처리 라이브러리
from sklearn.preprocessing import LabelEncoder, StandardScaler  # 범주형 데이터 인코딩, 스케일링
import joblib  # 객체를 직렬화하여 파일로 저장/불러오기
from sklearn.metrics import accuracy_score  # 정확도 계산을 위한 함수

# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        # 첫 번째 1D 컨볼루션 레이어 정의 (입력 채널: 1, 출력 채널: 32, 커널 크기: 3, 패딩: 1)
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        # 두 번째 1D 컨볼루션 레이어 정의 (입력 채널: 32, 출력 채널: 64, 커널 크기: 3, 패딩: 1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        # 최대 풀링 레이어 정의 (풀링 크기: 2)
        self.pool = nn.MaxPool1d(2)
        # 첫 번째 완전 연결층 (입력 크기: 64 * (input_size // 2), 출력 크기: 128)
        self.fc1 = nn.Linear(64 * (input_size // 2), 128)
        # 두 번째 완전 연결층 (입력 크기: 128, 출력 크기: num_classes)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Conv1D를 위해 배치 차원 뒤에 채널 차원을 추가
        x = x.unsqueeze(1)
        # 첫 번째 컨볼루션 + ReLU 활성화 함수
        x = F.relu(self.conv1(x))
        # 두 번째 컨볼루션 + ReLU 활성화 함수 후 최대 풀링
        x = self.pool(F.relu(self.conv2(x)))
        # 평탄화 (Flatten): 완전 연결층에 입력하기 위해 2D 형태로 변환
        x = x.view(x.size(0), -1)
        # 첫 번째 완전 연결층 + ReLU 활성화 함수
        x = F.relu(self.fc1(x))
        # 두 번째 완전 연결층: 예측값 출력
        x = self.fc2(x)
        return x


# 테스트 데이터 준비 함수
def tr_prepare_test_data(data_path):
    # CSV 파일 읽기
    data = pd.read_csv(data_path)
    
    # 'id' 열이 있다면 제거
    if 'id' in data.columns:
        data = data.drop('id', axis=1)
    
    # 'label' 열을 제외한 피처 정의 (레이블이 없으므로 'attack_cat' 제외)
    X = data.drop(['label'], axis=1, errors='ignore')  # 'label'이 존재하면 제외
    
    # 레이블 인코더와 스케일러 로드
    label_encoders = joblib.load('label_encoders.pkl')  # 저장된 레이블 인코더 불러오기
    scaler = joblib.load('scaler.pkl')  # 저장된 스케일러 불러오기
    
    # 인코딩할 카테고리 열 정의
    categorical_cols = ['proto', 'service', 'state', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd']
    
    # 'state' 열에서 'ACC'와 'CLO'를 가장 빈번한 값으로 대체
    if 'state' in X.columns:
        most_common_value = X['state'].mode()[0]  # 가장 빈번한 값 찾기
        X['state'] = X['state'].replace(['ACC', 'CLO'], most_common_value)
    
    # 카테고리 변수에 대한 인코딩 수행
    for col in categorical_cols:
        if col in X.columns:
            try:
                # 인코딩 수행
                X[col] = label_encoders[col].transform(X[col].astype(str))
            except ValueError as e:
                print(f"Error encoding {col}: {e}")
                # 에러 발생 시, 가장 빈번한 값으로 대체하고 재인코딩
                most_common_value = X[col].mode()[0]
                X[col] = X[col].fillna(most_common_value)  # NaN 값을 가장 빈번한 값으로 대체
                X[col] = label_encoders[col].transform(X[col].astype(str))
    
    # 피처 스케일링 수행
    X = scaler.transform(X)
    
    # X를 float32 타입으로 변환
    X = X.astype(np.float32)
    
    return X


# 테스트 정확도 계산 함수
def test(model, test_loader, device, true_labels=None):
    model.eval()  # 모델을 평가 모드로 설정
    predictions = []  # 예측 결과를 저장할 리스트
    
    # No gradient 계산 모드로 예측 수행
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch[0].to(device)  # 배치에서 X만 추출하여 장치로 이동
            outputs = model(X_batch)  # 모델 출력 (logits)
            _, predicted = torch.max(outputs, 1)  # 예측된 클래스 인덱스 추출
            predictions.extend(predicted.cpu().numpy())  # 예측값을 numpy 배열로 저장

    if true_labels is not None:
        # 실제 라벨이 제공될 경우, 정확도 계산
        acc = accuracy_score(true_labels, predictions)
        print(f"Test Accuracy: {acc:.4f}")
    else:
        # 실제 라벨 없이 예측값만 출력
        print("Predictions:", predictions)

    return predictions


if __name__ == '__main__':
    # 장치 설정 (GPU가 있으면 GPU 사용, 없으면 CPU 사용)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    filepath = './test-set.csv'  # 실제 라벨이 없는 테스트 파일 경로

    # 테스트 데이터 준비
    X_test = tr_prepare_test_data(filepath)  # 테스트 데이터 전처리 함수 호출
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # 텐서로 변환
    test_dataset = TensorDataset(X_test_tensor)  # 데이터셋 정의
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # DataLoader로 로딩

    # 모델 로드 (훈련된 모델이 존재한다고 가정)
    model = CNN(input_size=X_test.shape[1], num_classes=10).to(device)  # 클래스 수는 10개
    model.load_state_dict(torch.load('./model.pth', weights_only=True))  # 저장된 모델 가중치 로드

    # 테스트 데이터에서 예측 수행
    predictions = test(model, test_loader, device)

    # 예측된 공격 카테고리 매핑
    attack_categories = ['Normal', 'Fuzzers', 'Analysis', 'Backdoor', 'DoS', 'Exploits', 
                         'Generic', 'Reconnaissance', 'Shellcode', 'Worms']

    ids = range(1, len(predictions) + 1)  # ID는 1부터 시작

    # 예측 결과를 데이터프레임으로 저장
    prediction_df = pd.DataFrame({
        'id': ids,
        'attack_cat': [attack_categories[pred] for pred in predictions]  # 예측된 카테고리 매핑
    })

    # 예측 결과를 CSV 파일로 저장
    prediction_df.to_csv('submission_sample.csv', index=False)
    print("Predictions saved to 'submission_sample.csv'")

Predictions: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 5, 1, 8, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 6, 8, 1, 1, 7, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 1, 0, 1, 5, 0, 2, 2, 0, 2, 5, 7, 7, 0, 0, 2, 2, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 5, 0, 0, 1, 5, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 5, 1, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 8, 5, 5, 5, 7, 5, 4, 7, 7, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 7, 1, 1, 7, 1, 1, 5, 5, 7, 1, 0, 1, 1, 5, 5, 5, 0, 7, 0, 0, 1, 5, 0, 0, 5, 5, 7, 7, 5, 5, 5, 5, 5, 5, 1, 1, 0, 1, 0, 7, 7, 0, 1, 0, 5, 0, 0, 7, 0, 0, 7, 5, 5, 1, 1, 1, 1,

In [3]:
import pandas as pd  # 데이터프레임을 다루는 라이브러리 (CSV 파일 처리 등)
from sklearn.metrics import accuracy_score  # 정확도 계산을 위한 함수
# 실제 있는 데이터와 비교
submission_path = './submission_sample.csv'  # 예측 결과가 저장된 파일 경로
submission = pd.read_csv(submission_path)  # 예측 결과 CSV 파일을 읽어옴

true_labels_path = './UNSW_NB15_testing-set.csv'  # 실제 라벨이 포함된 테스트 데이터 파일 경로
true_labels = pd.read_csv(true_labels_path)  # 실제 라벨 CSV 파일을 읽어옴

# 'id'와 'attack_cat'을 기준으로 정확도 계산
# 'attack_cat' 열을 기준으로 실제 라벨(true_labels)과 예측 결과(submission) 간의 정확도를 계산
accuracy = accuracy_score(true_labels['attack_cat'], submission['attack_cat'])

# 계산된 정확도를 백분율로 출력
print(f"Test Accuracy: {accuracy * 100:.2f}%")  # 정확도를 100으로 곱해 백분율로 출력

C:\Users\gyxo0\anaconda3\envs\torch_ev\lib\site-packages\sklearn\utils\_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_pred contains NaN.